In [ ]:
# !pip install git+https://github.com/SethPoulsen/sklearn-crfsuite.git@add-missing-attributes#egg=sklearn_crfsuite

In [ ]:
# !pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2"

In [2]:
# import pandas as pd
import json
import sys
import copy


from itertools import chain

import sklearn
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [3]:
list_of_stc = []
list_of_list_token_of_stc = []
list_of_list_label_of_stc = []
list_of_list_POStag = []

list_of_sentiment = []
list_of_stc_id = []

with open("./file12345_isPOStag_NER_phase_filter_UNK_oversampling.txt", "r", encoding = "utf-8") as f:
    while f:
        info = f.readline()[:-1] # bỏ dấu xuống dòng
        stc = f.readline()[:-1]
        token = f.readline()[:-1]
        label = f.readline()[:-1]
        postag = f.readline()[:-1]

        # check hết file
        if info == "" and stc == "" and label == "":
            break

        # add idx, add sentiment
        info_split = info.split("\t")
        list_of_stc_id.append(int(info_split[0]))
        sentiment = json.loads(info_split[1])

        # add sentiment
        if sentiment[0] == 0:
            list_of_sentiment.append([1])
        elif sentiment[0] in [1, 2]:
            list_of_sentiment.append([2])
        else:
            assert sentiment[0] in [-1, -2]
            list_of_sentiment.append([0])

        # add stc, add label, add postag
        list_of_stc.append(json.loads(stc))
        list_of_list_token_of_stc.append(json.loads(token))
        list_of_list_label_of_stc.append(json.loads(label))
        list_of_list_POStag.append(json.loads(postag))

In [5]:
# Check how many labels are there in the dataset
unique_labels = set()

for list_lb in list_of_list_label_of_stc:
  [unique_labels.add(lb) for lb in list_lb if lb not in unique_labels]

print(unique_labels)

{'B-ap', 'O', 'I-ap', 'I-obj', 'B-obj', 'I-sub', 'B-sub'}


In [5]:
import random

def shuffle_data(data:list):
    random.shuffle(data)


def build_data_CEE_CRF(list_of_token_stc, list_of_POS_tag, list_CEE_label, is_unify_entity = False, is_shuffle_data = False):
    data_set = []
    CEE_label_dict = {
        "B-sub": "SUB",
        "I-sub": "SUB",
        "B-obj": "OBJ",
        "I-obj": "OBJ",
        "B-ap": "AP",
        "I-ap": "AP",
        "O": "O"
    }

    for index, CEE_label in enumerate(list_CEE_label):
        pos_tag = list_of_POS_tag[index]
        tokens = list_of_token_stc[index]
        
        assert len(pos_tag) == len(tokens)
        assert len(tokens) ==  len(CEE_label)

        list_for_stc = []
        for i, dict_ in enumerate(pos_tag):
            if is_unify_entity:
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[CEE_label[i]]]))
            else:
                assert CEE_label[i] in CEE_label_dict.keys()
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[i]]))
        data_set.append(list_for_stc)

    if is_shuffle_data:
        shuffle_data(data_set)
    end_train_idx = 9000
    return data_set[:end_train_idx], data_set[end_train_idx:]

data_NER_CRF_train, data_NER_CRF_test  = build_data_CEE_CRF(list_of_list_token_of_stc, list_of_list_POStag, list_of_list_label_of_stc, True)

In [8]:
data_NER_CRF_train[0]

[('Nhìn', 'V', 'O'),
 ('vào', 'E', 'O'),
 ('những', 'L', 'O'),
 ('thông_số', 'N', 'AP'),
 ('kích_thước', 'N', 'AP'),
 ('trên', 'N', 'O'),
 (',', 'CH', 'O'),
 ('VinFast_Lux_SA', 'Np', 'SUB'),
 ('2.0', 'M', 'SUB'),
 ('đang', 'R', 'O'),
 ('nhỉnh', 'V', 'O'),
 ('hơn', 'A', 'O'),
 ('Kia_Sorento', 'Np', 'OBJ'),
 ('về', 'E', 'O'),
 ('mọi', 'L', 'O'),
 ('chỉ_số', 'N', 'O'),
 (',', 'CH', 'O'),
 ('điều', 'N', 'O'),
 ('này', 'P', 'O'),
 ('giúp', 'V', 'O'),
 ('xe', 'N', 'O'),
 ('vạm_vỡ', 'V', 'O'),
 ('hơn', 'R', 'O'),
 ('khi', 'N', 'O'),
 ('nhìn', 'V', 'O'),
 ('từ', 'E', 'O'),
 ('phần', 'N', 'O'),
 ('thân', 'N', 'O')]

In [9]:
def _word2features_pos(sent, i, window):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:3]': postag[:3],
    }

    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            postag = sent[i - w][1]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
                '-{}:postag'.format(w): postag,
                '-{}:postag[:3]'.format(w): postag[:3],
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            postag1 = sent[i + w][1]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
                '+{}:postag'.format(w): postag1,
                '+{}:postag[:3]'.format(w): postag1[:3],
            })
        else:
            features['-{}:EOS'.format(w)] = True

    return features


def _word2features(sent, i, window):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
            })
        else:
            features['-{}:EOS'.format(w)] = True
    return features


def word2features(sent, i, is_pos, window):
    if is_pos:
        return _word2features_pos(sent, i, window)
    return _word2features(sent, i, window)

def sent2features(sent, is_pos, window):
    return [word2features(sent, i, is_pos, window) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent, is_pos):
    if is_pos:
        return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [10]:
for window_size in range(1,4):
    print(f'\n====================== Window_size : {window_size} ======================')
    print('POS-tag:')
    X_train = [sent2features(sent = s, is_pos = True, window = window_size) for s in data_NER_CRF_train]
    y_train = [sent2labels(sent = s) for s in data_NER_CRF_train]
    X_test = [sent2features(sent = s, is_pos = True, window = window_size) for s in data_NER_CRF_test]
    y_test = [sent2labels(sent = s) for s in data_NER_CRF_test]

    crf_model = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False,
    )

    crf_model.fit(X_train, y_train)
    pred = crf_model.predict(X_test)
    print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))
    del crf_model
    
    # ==========================================================================================================
    print('No POS-tag:')
    X_train = [sent2features(sent = s, is_pos = False, window = window_size) for s in data_NER_CRF_train]
    y_train = [sent2labels(sent = s) for s in data_NER_CRF_train]
    X_test = [sent2features(sent = s, is_pos = False, window = window_size) for s in data_NER_CRF_test]
    y_test = [sent2labels(sent = s) for s in data_NER_CRF_test]

    crf_model = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False,
    )

    crf_model.fit(X_train, y_train)
    pred = crf_model.predict(X_test)
    print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))
    del crf_model


====================== Window_size : 1 ======================
POS-tag:


/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

         SUB     0.8073    0.7770    0.7919       453
         OBJ     0.7057    0.6851    0.6953       343
          AP     0.7059    0.6176    0.6588       408

   micro avg     0.7451    0.6968    0.7202      1204
   macro avg     0.7396    0.6933    0.7153      1204
weighted avg     0.7440    0.6968    0.7193      1204

No POS-tag:


/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

         SUB     0.7955    0.7726    0.7839       453
         OBJ     0.7085    0.6589    0.6828       343
          AP     0.7143    0.5882    0.6452       408

   micro avg     0.7452    0.6777    0.7099      1204
   macro avg     0.7394    0.6733    0.7039      1204
weighted avg     0.7432    0.6777    0.7081      1204


====================== Window_size : 2 ======================
POS-tag:


...

/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

         SUB     0.8062    0.7439    0.7738       453
         OBJ     0.7609    0.6589    0.7063       343
          AP     0.6507    0.5980    0.6232       408

   micro avg     0.7404    0.6703    0.7036      1204
   macro avg     0.7393    0.6670    0.7011      1204
weighted avg     0.7406    0.6703    0.7035      1204


====================== Window_size : 3 ======================
POS-tag:


/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

         SUB     0.8146    0.7660    0.7895       453
         OBJ     0.7580    0.6939    0.7245       343
          AP     0.6921    0.5784    0.6302       408

   micro avg     0.7595    0.6819    0.7186      1204
   macro avg     0.7549    0.6794    0.7147      1204
weighted avg     0.7569    0.6819    0.7170      1204

No POS-tag:
              precision    recall  f1-score   support

         SUB     0.7928    0.7770    0.7848       453
         OBJ     0.7438    0.6939    0.7179       343
          AP     0.7121    0.5760    0.6369       408

   micro avg     0.7541    0.6852    0.7180      1204
   macro avg     0.7496    0.6823    0.7132      1204
weighted avg     0.7515    0.6852    0.7156      1204



/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
